In [1]:
import pandas as pd
import json

In [25]:
flatten = lambda x: [j for i in x for j in i]

In [80]:
with open("../../data/pageID2CID_using_wikidata.json", 'r') as f:
    pageID2CID_table = json.load(f)

with open("../../data/compound_train.json", 'r') as f:
    train_raw = json.load(f)
    train_product = {str(entry['WikipediaID']): {'Name': entry['Name'], 'Product': entry['Attributes']['製造方法']} for entry in train_raw['entry']}
    pageID_set = set(pageID2title_table.keys())

with open("../../data/raw_manufacturings_using_wikidata.json", 'r') as f:
    manufacturings = json.load(f)

In [64]:
wiki_df = pd.read_csv("../../data/train_split_words.csv")
wiki_df = wiki_df.append(pd.read_csv("../../data/valid_split_words.csv"))
wiki_df._id = wiki_df._id.astype(str)

In [172]:
with open("../../data/pageID2wikidataID.json", 'r') as f:
    pageid2wikidataid_table = json.load(f)

In [175]:
len(pageid2wikidataid_table)

5456

In [61]:
print(len(manufacturings))
print(len(manufacturing_df[manufacturing_df._id.isin(pageID_set)]._id.unique()))

4490
211


In [35]:
train_contained_prod = [str(entry['WikipediaID']) for entry in train_raw['entry'] if entry['Attributes']['製造方法']]

In [37]:
train_notcontained_prod = [str(entry['WikipediaID']) for entry in train_raw['entry'] if not entry['Attributes']['製造方法']]

In [71]:
true_pageID_set = set(wiki_df[wiki_df.label == True]._id.unique())
len(true_pageID_set)

305

In [76]:
len(set(train_contained_prod) - true_pageID_set)

17

In [89]:
wiki_df[wiki_df._id.isin(set(train_contained_prod) - true_pageID_set)][['_id', 'title', 'sentence']].values

array([['1533538', 'ペンタニトロアニリン',
        'ペンタニトロアニリン (pentanitroaniline) は、アニリンのベンゼン環上にある5個の水素がすべてニトロ基に置き換わった構造を持つ芳香族ニトロ化合物である。'],
       ['1533538', 'ペンタニトロアニリン', '爆薬、推進薬として検討の対象となった。'],
       ['1533538', 'ペンタニトロアニリン', '3,5-ジニトロ-p-トルイジンに混酸を作用させて合成した例がある。'],
       ['1533538', 'ペンタニトロアニリン', 'この反応では基質のメチル基がイプソ置換を受けニトロ基に置換される。'],
       ['841166', 'アミドキシム', 'オキシアミジンは化合物の一種。'],
       ['1383393', '炭酸銅(II)',
        '炭酸銅(II)（たんさんどう に、英: copper(II) carbonate）は、銅の炭酸塩である。'],
       ['1383393', '炭酸銅(II)',
        '単に炭酸銅という時には、２価の銅イオン Cu2+ と炭酸イオン CO32− と水酸化物イオン OH− から成る塩基性炭酸銅を指すことが多い。'],
       ['1383393', '炭酸銅(II)',
        '炭酸銅(II)の正塩 CuCO3 は知られていない、とする辞典もあるが、高温高圧のCO2雰囲気下で三方晶系または単斜晶系の CuCO3 を塩基性塩から合成できる、という報告もある。'],
       ['145521', 'ヘキサメチレンジアミン',
        'ヘキサメチレンジアミン (hexamethylene diamine) は示性式 H2N-(CH2)6-NH2、分子量 116.20 のジアミン。'],
       ['145521', 'ヘキサメチレンジアミン', 'ピペリジン様の臭気のある、吸湿性の白色固体。'],
       ['145521', 'ヘキサメチレンジアミン', '融点は 42 ℃、沸点は 204 ℃。'],
       ['145521', 'ヘキサメチレンジアミン',
        'IU

In [63]:
len(train_contained_prod)

322

In [39]:
len(train_notcontained_prod)

276

In [169]:
# Wikipediaに製造方法がある & PubChem対応
rel1 = set([page_id for page_id in train_contained_prod if pageID2CID_table.get(page_id)])
print(len(rel1))

# Wikipediaに製造方法がある & PubChem非対応
rel2 = set([page_id for page_id in train_contained_prod if  pageID2CID_table.get(page_id) is None])
print(len(rel2))

# Wikipediaに製造方法がない & PubChemに対応
rel3 = set([page_id for page_id in train_notcontained_prod if pageID2CID_table.get(page_id)])
print(len(rel3))

# Wikipediaに製造方法がない & PubChem非対応
rel4 = set([page_id for page_id in train_notcontained_prod if pageID2CID_table.get(page_id) is None])
print(len(rel4))

253
69
200
76


In [170]:
len(manufacturing_df[manufacturing_df._id.isin(rel3)]._id.unique())

93

In [171]:
{page_id: train_product[page_id] 
 #for page_id in manufacturing_df[manufacturing_df._id.isin(rel2)]._id.unique()
 for page_id in rel2
}

{'2639020': {'Name': 'オルト過ヨウ素酸ナトリウム', 'Product': ['ヨウ素酸ナトリウムを分解する']},
 '1614092': {'Name': '鉄酸バリウム',
  'Product': ['鉄酸カリウムと塩化バリウムを含む溶液から溶解しないBaFeO4の沈殿によって得られる']},
 '3609064': {'Name': 'アニスアルコール',
  'Product': ['p-アニスアルデヒドを、白金触媒を用いて水素化（還元）する',
   'p-アニスアルデヒドを、ヒドリド還元剤で還元する',
   'p-アニスアルデヒドに対して、カニッツァーロ反応を起こす。すなわち、p-アニスアルデヒドに適切な塩基を作用させて、p-アニス酸とアニスアルコールとに不均化させる方法である']},
 '3692832': {'Name': '4-メルカプト-4-メチル-2-ペンタノン', 'Product': ['煎茶 の火入れ工程で発生']},
 '3472884': {'Name': 'REBCO',
  'Product': ['電子ビーム共蒸着法',
   'レーザアブレーション(PLD)法',
   '有機金属化学気相蒸着法',
   '原料槽に入れた金属錯体原料をヒーターで加熱し液体状態にし、キャリアガスを原料槽内に流通することにより、気化した原料ガスを反応室へと導き、セラミックヒーターで加熱された基板上に蒸着させ成膜を行う。',
   'MOCVD',
   'PLD法',
   '原料槽に入れた金属錯体原料をヒーターで加熱し液体状態にし、キャリアガスを原料槽内に流通することにより、気化した原料ガスを反応室へと導き、セラミックヒーターで加熱された基板上に蒸着させ成膜を行う',
   '真空容器内で複数の蒸着源を個々に電子ビームで加熱蒸発させ、ヒーターによって加熱された基板上に薄膜として成長させる',
   '真空チャンバー内の焼結体ターゲットにパルスレーザを断続的に照射してターゲットをアブレーションすることにより放出されるフラグメント（イオン、クラスタ、分子、原子）をターゲットと対向して配置された基板上に薄膜を堆積させる']},
 '3526170': {'Name': 'リン酸三カルシウム', 'Product': ['骨